# * VINSIGHT : Subscriber
    กลุ่ม KPIs ของ Postpaid Reported SubBase B2C : TRUE มาจากพี่เอ้รัน scripts พี่หนุ่ย (SubBase, 60DPD, Gain/Loss 60DPD)
    กลุ่ม KPIs ของ Postpaid Reported SubBase B2C : DTAC ยังมาจากพี่เอกรันให้อยู่ (SubBase, 60DPD, Gain/Loss 60DPD)
    กลุ่ม KPIs ของ FTTx Reported SubBase ยังมาจากพี่เอกรันให้อยู่ (SubBase, 60DPD, Gain/Loss 60DPD)
    Postpaid Reported SubBase B2C เริ่มตั้งแต่ June2024++
    Prepaid Active Caller : TMH เริ่มตั้งแต่ May2024++

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000500' --Prepaid Usage Subs
        , 'TB1S000500' --Prepaid Usage Subs : TMH
        , 'DB1S000500' --Prepaid Usage Subs : DTAC
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'TB1S000600' --Prepaid Active Caller 30D Rolling : TMH
        , 'DB1S000600' --Prepaid Active Caller 30D Rolling : DTAC
        , 'B1S000702' --Prepaid SubBase
        , 'DB1S000702' --Prepaid SubBase : DTAC
        , 'TB1S000702' --Prepaid SubBase : TMH
        , 'B1S000700' --Prepaid Reported SubBase
        , 'DB1S000700' --Prepaid Reported SubBase : DTAC
        , 'TB1S000700' --Prepaid Reported SubBase : TMH
        
        , 'B2S010602' --Postpaid SubBase B2C
        , 'DB2S010602' --Postpaid SubBase B2C : DTAC
        , 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        
        , 'TB3S020604' --FTTx SubBase
        , 'TB3S000600' --FTTx Reported SubBase

        , 'TB4S000500' --TVS Active Subs
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-07-15, 10:35:58

DataFrame: 7854 rows, 16 columns


In [3]:
''' Automate Currently Period '''

curr_yr = chk_src_df['TM_KEY_YR'].max()
prev_yr = chk_src_df['TM_KEY_YR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_qtr = chk_src_df['TM_KEY_QTR'].max()
prev_qtr = chk_src_df['TM_KEY_QTR'].drop_duplicates().sort_values().shift().max().astype(int)

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = chk_src_df['TM_KEY_MTH'].drop_duplicates().sort_values().shift().max().astype(int)

curr_wk = chk_src_df['TM_KEY_WK'].max()
prev_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift().max().astype(int)
last_3_wk = chk_src_df['TM_KEY_WK'].drop_duplicates().sort_values().shift(3).max().astype(int)

# curr_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].max().astype(int)
# prev_day = chk_src_df.loc[chk_src_df['ACTUAL_SNAP'] > 0]['ACTUAL_AS_OF'].drop_duplicates().shift().max().astype(int)

## Overview by Period

In [4]:
''' Subscriber Yearly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_yearly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_YR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_YR
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
sub_yearly_df = sub_yearly_df.loc[sub_yearly_df['METRIC_CD'].isin(v_metric_list)]

sub_yearly_df['PRE_CALL_30D'] = np.where(sub_yearly_df['METRIC_CD']=='B1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['PRE_CALL_30D_T'] = np.where(sub_yearly_df['METRIC_CD']=='TB1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['PRE_CALL_30D_D'] = np.where(sub_yearly_df['METRIC_CD']=='DB1S000600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C'] = np.where(sub_yearly_df['METRIC_CD']=='B2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_yearly_df['METRIC_CD']=='TB2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_yearly_df['METRIC_CD']=='DB2S010600', sub_yearly_df['P'], 0)
sub_yearly_df['TOL_RPT_SUB'] = np.where(sub_yearly_df['METRIC_CD']=='TB3S000600', sub_yearly_df['P'], 0)
sub_yearly_df['TVS_ACTV_SUB'] = np.where(sub_yearly_df['METRIC_CD']=='TB4S000500', sub_yearly_df['P'], 0)

sub_yearly_df = sub_yearly_df.groupby('TM_KEY_YR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_yearly_df = sub_yearly_df.fillna(0).sort_values(by=['TM_KEY_YR']).reset_index()
sub_yearly_df = sub_yearly_df[['TM_KEY_YR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_yearly_df_display = sub_yearly_df.copy()
sub_yearly_df_display['ACTUAL_AS_OF'] = sub_yearly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_yearly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_yearly_df_display[col] = sub_yearly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_yearly_df_display

,TM_KEY_YR,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,2024,2025-07-15 04:56:33,20241231,"28,100,141","17,769,553","10,330,588","14,158,698","9,102,725","5,055,973","3,154,889","1,180,410"
1,2025,2025-07-15 04:56:33,20250713,"26,078,802","16,204,177","9,874,625","14,025,276","9,084,691","4,944,042","3,208,434","1,104,655"


In [5]:
''' Subscriber Quarterly '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_quarterly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_QTR', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_QTR
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']]
sub_quarterly_df = sub_quarterly_df.loc[sub_quarterly_df['METRIC_CD'].isin(v_metric_list)]

sub_quarterly_df['PRE_CALL_30D'] = np.where(sub_quarterly_df['METRIC_CD']=='B1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['PRE_CALL_30D_T'] = np.where(sub_quarterly_df['METRIC_CD']=='TB1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['PRE_CALL_30D_D'] = np.where(sub_quarterly_df['METRIC_CD']=='DB1S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C'] = np.where(sub_quarterly_df['METRIC_CD']=='B2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_quarterly_df['METRIC_CD']=='TB2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_quarterly_df['METRIC_CD']=='DB2S010600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TOL_RPT_SUB'] = np.where(sub_quarterly_df['METRIC_CD']=='TB3S000600', sub_quarterly_df['P'], 0)
sub_quarterly_df['TVS_ACTV_SUB'] = np.where(sub_quarterly_df['METRIC_CD']=='TB4S000500', sub_quarterly_df['P'], 0)

sub_quarterly_df = sub_quarterly_df.groupby('TM_KEY_QTR').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_quarterly_df = sub_quarterly_df.fillna(0).sort_values(by=['TM_KEY_QTR']).reset_index()
sub_quarterly_df = sub_quarterly_df[['TM_KEY_QTR', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_quarterly_df_display = sub_quarterly_df.copy()
sub_quarterly_df_display['ACTUAL_AS_OF'] = sub_quarterly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_quarterly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_quarterly_df_display[col] = sub_quarterly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_quarterly_df_display

,TM_KEY_QTR,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,20241,2025-07-15 04:56:33,20240331,0,0,"10,558,566",0,"8,994,853",0,"3,090,287","1,258,065"
1,20242,2025-07-15 04:56:33,20240630,"27,800,517","17,313,488","10,487,029","14,226,167","9,042,184","5,183,983","3,107,084","1,236,606"
2,20243,2025-07-15 04:56:33,20240930,"27,505,003","17,190,424","10,314,579","14,186,615","9,068,470","5,118,145","3,127,304","1,205,116"
3,20244,2025-07-15 04:56:33,20241231,"28,100,141","17,769,553","10,330,588","14,158,698","9,102,725","5,055,973","3,154,889","1,180,410"
4,20251,2025-07-15 04:56:33,20250331,"27,140,083","17,215,465","9,924,618","14,084,812","9,086,820","4,997,992","3,187,717","1,153,137"
5,20252,2025-07-15 04:56:33,20250630,"26,272,750","16,372,729","9,900,021","14,132,231","9,179,698","4,952,533","3,207,941","1,112,829"
6,20253,2025-07-15 04:56:33,20250713,"26,078,802","16,204,177","9,874,625","14,025,276","9,084,691","4,944,042","3,208,434","1,104,655"


In [6]:
''' Subscriber Monthly '''

# v_tm_key_yr = curr_yr
v_tm_key_yr = prev_yr

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_YR']>=v_tm_key_yr]
sub_monthly_df = sub_monthly_df.loc[sub_monthly_df['METRIC_CD'].isin(v_metric_list)]

sub_monthly_df['PRE_CALL_30D'] = np.where(sub_monthly_df['METRIC_CD']=='B1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['PRE_CALL_30D_T'] = np.where(sub_monthly_df['METRIC_CD']=='TB1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['PRE_CALL_30D_D'] = np.where(sub_monthly_df['METRIC_CD']=='DB1S000600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C'] = np.where(sub_monthly_df['METRIC_CD']=='B2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_monthly_df['METRIC_CD']=='TB2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_monthly_df['METRIC_CD']=='DB2S010600', sub_monthly_df['P'], 0)
sub_monthly_df['TOL_RPT_SUB'] = np.where(sub_monthly_df['METRIC_CD']=='TB3S000600', sub_monthly_df['P'], 0)
sub_monthly_df['TVS_ACTV_SUB'] = np.where(sub_monthly_df['METRIC_CD']=='TB4S000500', sub_monthly_df['P'], 0)

sub_monthly_df = sub_monthly_df.groupby('TM_KEY_MTH').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_monthly_df = sub_monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH']).reset_index()
sub_monthly_df = sub_monthly_df[['TM_KEY_MTH', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_monthly_df_display = sub_monthly_df.copy()
sub_monthly_df_display['ACTUAL_AS_OF'] = sub_monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_monthly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_monthly_df_display[col] = sub_monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_monthly_df_display

,TM_KEY_MTH,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,202401,2025-07-15 04:56:33,20240131,0,0,"10,612,127",0,0,0,0,"1,280,972"
1,202402,2025-07-15 04:56:33,20240229,0,0,"10,729,197",0,"8,998,482",0,"3,084,265","1,269,692"
2,202403,2025-07-15 04:56:33,20240331,0,0,"10,558,566",0,"8,994,853",0,"3,090,287","1,258,065"
3,202404,2025-07-15 04:56:33,20240430,0,0,"10,637,624",0,"8,996,646",0,"3,091,767","1,249,008"
4,202405,2025-07-15 04:56:33,20240531,"28,022,052","17,391,242","10,630,810",0,"9,023,324",0,"3,098,514","1,245,639"
5,202406,2025-07-15 04:56:33,20240630,"27,800,517","17,313,488","10,487,029","14,226,167","9,042,184","5,183,983","3,107,084","1,236,606"
6,202407,2025-07-15 04:56:33,20240731,"27,805,282","17,342,927","10,462,355","14,209,033","9,052,569","5,156,464","3,114,835","1,226,581"
7,202408,2025-07-15 04:56:33,20240831,"27,664,960","17,247,805","10,417,155","14,190,431","9,061,002","5,129,429","3,122,029","1,214,582"
8,202409,2025-07-15 04:56:33,20240930,"27,505,003","17,190,424","10,314,579","14,186,615","9,068,470","5,118,145","3,127,304","1,205,116"
9,202410,2025-07-15 04:56:33,20241031,"27,425,422","17,195,820","10,229,602","14,179,406","9,080,143","5,099,263","3,134,367","1,195,552"


In [7]:
''' Subscriber Weekly '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_weekly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_WK', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_WK
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF', 'P']].loc[chk_src_df['TM_KEY_MTH']>=prev_mth]
sub_weekly_df = sub_weekly_df.loc[sub_weekly_df['METRIC_CD'].isin(v_metric_list)]

sub_weekly_df['PRE_CALL_30D'] = np.where(sub_weekly_df['METRIC_CD']=='B1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['PRE_CALL_30D_T'] = np.where(sub_weekly_df['METRIC_CD']=='TB1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['PRE_CALL_30D_D'] = np.where(sub_weekly_df['METRIC_CD']=='DB1S000600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C'] = np.where(sub_weekly_df['METRIC_CD']=='B2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C_T'] = np.where(sub_weekly_df['METRIC_CD']=='TB2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['POST_RPT_SUB_B2C_D'] = np.where(sub_weekly_df['METRIC_CD']=='DB2S010600', sub_weekly_df['P'], 0)
sub_weekly_df['TOL_RPT_SUB'] = np.where(sub_weekly_df['METRIC_CD']=='TB3S000600', sub_weekly_df['P'], 0)
sub_weekly_df['TVS_ACTV_SUB'] = np.where(sub_weekly_df['METRIC_CD']=='TB4S000500', sub_weekly_df['P'], 0)

sub_weekly_df = sub_weekly_df.groupby('TM_KEY_WK').agg({'PPN_TM':'max', 'ACTUAL_AS_OF':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_weekly_df = sub_weekly_df.fillna(0).sort_values(by=['TM_KEY_WK']).reset_index()
sub_weekly_df = sub_weekly_df[['TM_KEY_WK', 'PPN_TM', 'ACTUAL_AS_OF', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_weekly_df_display = sub_weekly_df.copy()
sub_weekly_df_display['ACTUAL_AS_OF'] = sub_weekly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = sub_weekly_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_weekly_df_display[col] = sub_weekly_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_weekly_df_display

,TM_KEY_WK,PPN_TM,ACTUAL_AS_OF,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,2025022,2025-07-15 04:56:33,20250601,"26,438,734","16,530,315","9,908,419","14,073,719","9,106,502","4,967,217","3,205,472","1,128,753"
1,2025023,2025-07-15 04:56:33,20250608,"26,362,107","16,469,398","9,892,709","14,064,757","9,102,669","4,962,088","3,202,722","1,126,877"
2,2025024,2025-07-15 04:56:33,20250615,"26,318,308","16,463,082","9,855,226","14,045,663","9,090,429","4,955,234","3,201,297","1,119,850"
3,2025025,2025-07-15 04:56:33,20250622,"26,199,621","16,389,947","9,809,674","14,044,858","9,088,932","4,955,926","3,204,117","1,117,130"
4,2025026,2025-07-15 04:56:33,20250629,"26,257,033","16,363,001","9,894,032","14,031,799","9,081,416","4,950,383","3,204,018","1,113,416"
5,2025027,2025-07-15 04:56:33,20250706,"26,234,654","16,339,704","9,894,950","14,033,281","9,081,665","4,951,616","3,203,793","1,109,378"
6,2025028,2025-07-15 04:56:33,20250713,"26,078,802","16,204,177","9,874,625","14,025,276","9,084,691","4,944,042","3,208,434","1,104,655"


In [8]:
''' Subscriber Daily '''

v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_T'] = np.where(sub_daily_df['METRIC_CD']=='TB1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_D'] = np.where(sub_daily_df['METRIC_CD']=='DB1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_T'] = np.where(sub_daily_df['METRIC_CD']=='TB2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_D'] = np.where(sub_daily_df['METRIC_CD']=='DB2S010600', sub_daily_df['P'], 0)
sub_daily_df['TOL_RPT_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY'], ascending=False).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

sub_daily_df_display = sub_daily_df.copy()
mod_col_list = sub_daily_df_display.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df_display[col] = sub_daily_df_display[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df_display

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,202507,20250714,2025-07-15 04:56:33,0,0,0,0,0,0,0,0
1,202507,20250713,2025-07-15 04:56:33,"26,078,802","16,204,177","9,874,625",0,"9,084,691",0,0,"1,104,655"
2,202507,20250712,2025-07-15 04:56:33,"26,091,180","16,207,586","9,883,594","14,025,276","9,081,234","4,944,042",0,"1,105,528"
3,202507,20250711,2025-07-15 04:56:33,"26,089,515","16,201,042","9,888,473","14,030,663","9,087,898","4,942,765","3,208,434","1,105,830"
4,202507,20250710,2025-07-15 04:56:33,"26,083,156","16,206,888","9,876,268","14,033,046","9,083,650","4,949,396","3,206,564","1,106,376"
5,202507,20250709,2025-07-15 04:56:33,"9,887,298",0,"9,887,298","4,947,866",0,"4,947,866","3,205,624","1,107,655"
6,202507,20250708,2025-07-15 04:56:33,"26,170,128","16,274,115","9,896,013","14,033,690","9,087,187","4,946,503","3,207,719","1,107,849"
7,202507,20250707,2025-07-15 04:56:33,"26,206,133","16,317,126","9,889,007","14,030,798","9,084,386","4,946,412","3,205,789","1,109,378"
8,202507,20250706,2025-07-15 04:56:33,"26,234,654","16,339,704","9,894,950","14,033,281","9,081,665","4,951,616","3,203,793","1,109,378"
9,202507,20250705,2025-07-15 04:56:33,"26,254,683","16,354,285","9,900,398","14,038,734","9,088,110","4,950,624","3,207,918","1,109,696"


## Products Summary

### Prep Monthly Data

In [9]:
''' Monthly Summary '''

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
monthly_df_display['ACTUAL_AS_OF'] = monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))

# monthly_df_display
# monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]

In [10]:
''' Parameter '''

# v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth
v_tm_key_mth = 202501

print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202501


### Prepaid

In [11]:
''' Prepaid Usage Subs '''

v_metric_list = [
	'B1S000500' #Prepaid Usage Subs
    , 'TB1S000500' #Prepaid Usage Subs : TMH
    , 'DB1S000500' #Prepaid Usage Subs : DTAC
    ]

prepaid_usage_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_usage_sub_df = prepaid_usage_sub_df.loc[prepaid_usage_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_usage_sub_df = prepaid_usage_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
# prepaid_usage_sub_df = prepaid_usage_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_usage_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)


In [12]:
''' Prepaid Active Caller 30D Rolling '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    ]

prepaid_call_30d_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_call_30d_df = prepaid_call_30d_df.loc[prepaid_call_30d_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_call_30d_df = prepaid_call_30d_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_call_30d_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-07-15 04:56:33,20250131,"27,745,939","27,745,939","27,731,832","27,731,832","27,731,832",0
1,202501,Prepaid,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-07-15 04:56:33,20250131,"10,401,645","10,401,645","10,400,923","10,400,923","10,400,923",0
2,202501,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20250131,"17,344,294","17,344,294","17,330,909","17,330,909","17,330,909",0
3,202502,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-07-15 04:56:33,20250228,"27,605,672","27,605,672","27,587,202","27,587,202","27,587,202",0
4,202502,Prepaid,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-07-15 04:56:33,20250228,"10,180,996","10,180,996","10,179,777","10,179,777","10,179,777",0
5,202502,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20250228,"17,424,676","17,424,676","17,407,425","17,407,425","17,407,425",0
6,202503,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-07-15 04:56:33,20250331,"27,140,083","27,140,083","27,119,364","27,119,364","27,119,364",0
7,202503,Prepaid,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-07-15 04:56:33,20250331,"9,924,618","9,924,618","9,923,719","9,923,719","9,923,719",0
8,202503,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20250331,"17,215,465","17,215,465","17,195,645","17,195,645","17,195,645",0
9,202504,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-07-15 04:56:33,20250430,"26,599,795","26,599,795","26,578,640","26,578,640","26,578,640",0


In [13]:
''' Prepaid SubBase '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
    ]

prepaid_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_subbase_df = prepaid_subbase_df.loc[prepaid_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_subbase_df = prepaid_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-07-15 04:56:33,20250131,0,"24,942,269","24,769,264","24,267,797","24,769,149","-24,942,269"
1,202502,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-07-15 04:56:33,20250228,0,"24,921,942","24,760,083","24,758,973","24,758,973","-24,921,942"
2,202503,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-07-15 04:56:33,20250331,0,"24,578,672","24,437,486","24,436,544","24,436,544","-24,578,672"
3,202504,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-07-15 04:56:33,20250430,0,"23,721,266","23,582,316","23,581,524","23,581,524","-23,721,266"
4,202505,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-07-15 04:56:33,20250531,0,"23,721,160","23,561,656","23,560,980","23,560,980","-23,721,160"
5,202506,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-07-15 04:56:33,20250630,0,"22,959,730","22,840,094","22,839,500","22,839,500","-22,959,730"
6,202507,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-07-15 04:56:33,20250712,0,"22,330,665","22,225,025","22,224,456","22,224,456","-22,330,665"


In [14]:
''' Prepaid Reported SubBase '''

v_metric_list = [
    'B1S000700' #Prepaid Reported SubBase
    , 'DB1S000700' #Prepaid Reported SubBase : DTAC
    , 'TB1S000700' #Prepaid Reported SubBase : TMH
    ]

prepaid_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_report_sub_df = prepaid_report_sub_df.loc[prepaid_report_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
prepaid_report_sub_df = prepaid_report_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
prepaid_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-07-15 04:56:33,20250131,0,"21,379,455","21,262,196","21,262,196","21,262,196","-21,379,455"
1,202502,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-07-15 04:56:33,20250228,0,"21,170,310","21,078,525","21,078,525","21,078,525","-21,170,310"
2,202503,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-07-15 04:56:33,20250331,0,"20,647,169","20,559,792","20,559,792","20,559,792","-20,647,169"
3,202504,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-07-15 04:56:33,20250430,0,"20,304,003","20,209,194","20,209,194","20,209,194","-20,304,003"
4,202505,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-07-15 04:56:33,20250531,0,"20,108,592","19,961,598","19,961,598","19,961,598","-20,108,592"
5,202506,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-07-15 04:56:33,20250630,0,"19,640,822","19,508,349","19,508,349","19,508,349","-19,640,822"
6,202507,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-07-15 04:56:33,20250713,0,"19,390,221","19,282,475","19,282,475","19,282,475","-19,390,221"


### Postpaid

In [15]:
''' Postpaid SubBase B2C '''

v_metric_list = [
    'B2S010602' #Postpaid SubBase B2C
    , 'DB2S010602' #Postpaid SubBase B2C : DTAC
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
    ]

postpaid_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.loc[postpaid_subbase_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Postpaid,B2S010602,Postpaid SubBase B2C,2025-07-15 04:56:33,20250131,0,"14,274,718","14,274,477","14,268,794","14,268,794","-14,274,718"
1,202501,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-07-15 04:56:33,20250131,0,"5,161,625","5,161,620","5,155,964","5,155,964","-5,161,625"
2,202501,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-07-15 04:56:33,20250131,0,"9,113,093","9,112,857","9,112,830","9,112,830","-9,113,093"
3,202502,Postpaid,B2S010602,Postpaid SubBase B2C,2025-07-15 04:56:33,20250228,0,"14,273,218","14,272,963","14,267,293","14,267,293","-14,273,218"
4,202502,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-07-15 04:56:33,20250228,0,"5,138,667","5,138,664","5,133,010","5,133,010","-5,138,667"
5,202502,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-07-15 04:56:33,20250228,0,"9,134,551","9,134,299","9,134,283","9,134,283","-9,134,551"
6,202503,Postpaid,B2S010602,Postpaid SubBase B2C,2025-07-15 04:56:33,20250331,0,"14,288,650","14,288,393","14,282,728","14,282,728","-14,288,650"
7,202503,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-07-15 04:56:33,20250331,0,"5,118,045","5,118,040","5,112,387","5,112,387","-5,118,045"
8,202503,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-07-15 04:56:33,20250331,0,"9,170,605","9,170,353","9,170,341","9,170,341","-9,170,605"
9,202504,Postpaid,B2S010602,Postpaid SubBase B2C,2025-07-15 04:56:33,20250430,0,"14,301,830","14,301,541","14,295,878","14,295,878","-14,301,830"


In [16]:
''' Postpaid Reported SubBase B2C '''

v_metric_list = [
    'B2S010600' #Postpaid Reported SubBase B2C
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    ]

postpaid_report_sub_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.loc[postpaid_report_sub_b2c_df['TM_KEY_MTH']>=v_tm_key_mth]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
postpaid_report_sub_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20250131,0,"14,069,220","14,063,387","13,889,946","14,063,360","-14,069,220"
1,202501,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20250131,0,"5,033,939","5,028,342","4,854,928","5,028,342","-5,033,939"
2,202501,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20250131,0,"9,035,281","9,035,045","9,035,018","9,035,018","-9,035,281"
3,202502,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20250228,0,"14,068,955","14,063,110","14,063,094","14,063,094","-14,068,955"
4,202502,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20250228,0,"5,013,920","5,008,326","5,008,326","5,008,326","-5,013,920"
5,202502,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20250228,0,"9,055,035","9,054,784","9,054,768","9,054,768","-9,055,035"
6,202503,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20250331,0,"14,084,812","14,078,967","14,078,955","14,078,955","-14,084,812"
7,202503,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20250331,0,"4,997,992","4,992,399","4,992,399","4,992,399","-4,997,992"
8,202503,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20250331,0,"9,086,820","9,086,568","9,086,556","9,086,556","-9,086,820"
9,202504,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20250430,0,"14,100,846","14,094,971","14,094,960","14,094,960","-14,100,846"


### TOL

In [17]:
''' FTTx SubBase '''

v_metric_list = ['TB3S020604'] #FTTx SubBase

fttx_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_subbase_df = fttx_subbase_df.loc[fttx_subbase_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_subbase_df = fttx_subbase_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,TOL,TB3S020604,FTTx SubBase,2025-07-15 04:56:33,20250131,0,"3,280,710","3,280,625","3,280,625","3,280,625","-3,280,710"
1,202502,TOL,TB3S020604,FTTx SubBase,2025-07-15 04:56:33,20250228,0,"3,290,380","3,290,298","3,290,298","3,290,298","-3,290,380"
2,202503,TOL,TB3S020604,FTTx SubBase,2025-07-15 04:56:33,20250331,0,"3,305,177","3,305,095","3,305,095","3,305,095","-3,305,177"
3,202504,TOL,TB3S020604,FTTx SubBase,2025-07-15 04:56:33,20250430,0,"3,315,573","3,315,491","3,315,491","3,315,491","-3,315,573"
4,202505,TOL,TB3S020604,FTTx SubBase,2025-07-15 04:56:33,20250531,0,"3,294,820","3,294,739","3,294,739","3,294,739","-3,294,820"
5,202506,TOL,TB3S020604,FTTx SubBase,2025-07-15 04:56:33,20250630,0,"3,304,012","3,303,931","3,303,931","3,303,931","-3,304,012"
6,202507,TOL,TB3S020604,FTTx SubBase,2025-07-15 04:56:33,20250711,0,"3,309,370","3,309,291","3,309,291","3,309,291","-3,309,370"


In [18]:
''' FTTx Reported SubBase '''

v_metric_list = ['TB3S000600'] #FTTx Reported SubBase

fttx_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_report_sub_df = fttx_report_sub_df.loc[fttx_report_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
fttx_report_sub_df = fttx_report_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
fttx_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,TOL,TB3S000600,FTTx Reported SubBase,2025-07-15 04:56:33,20250131,0,"3,163,192","3,163,109","3,163,109","3,163,109","-3,163,192"
1,202502,TOL,TB3S000600,FTTx Reported SubBase,2025-07-15 04:56:33,20250228,0,"3,172,770","3,172,690","3,172,690","3,172,690","-3,172,770"
2,202503,TOL,TB3S000600,FTTx Reported SubBase,2025-07-15 04:56:33,20250331,0,"3,187,717","3,187,637","3,187,637","3,187,637","-3,187,717"
3,202504,TOL,TB3S000600,FTTx Reported SubBase,2025-07-15 04:56:33,20250430,0,"3,191,503","3,191,424","3,191,424","3,191,424","-3,191,503"
4,202505,TOL,TB3S000600,FTTx Reported SubBase,2025-07-15 04:56:33,20250531,0,"3,203,127","3,203,048","3,203,048","3,203,048","-3,203,127"
5,202506,TOL,TB3S000600,FTTx Reported SubBase,2025-07-15 04:56:33,20250630,0,"3,207,941","3,207,862","3,207,862","3,207,862","-3,207,941"
6,202507,TOL,TB3S000600,FTTx Reported SubBase,2025-07-15 04:56:33,20250711,0,"3,208,434","3,208,355","3,208,355","3,208,355","-3,208,434"


### TVS

In [19]:
''' TVS Active Subs '''

v_metric_list = ['TB4S000500'] #TVS Active Subs

tvs_active_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
tvs_active_sub_df = tvs_active_sub_df.loc[tvs_active_sub_df['TM_KEY_MTH']>=v_tm_key_mth]
tvs_active_sub_df = tvs_active_sub_df.sort_values(by=['TM_KEY_MTH', 'METRIC_CD']).reset_index(drop=True)
tvs_active_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202501,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,20250131,"1,191,955","1,170,444","1,170,444","1,170,444","1,170,444","21,511"
1,202502,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,20250228,"1,185,388","1,163,299","1,163,299","1,163,299","1,163,299","22,089"
2,202503,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,20250331,"1,175,395","1,153,137","1,153,137","1,153,137","1,153,137","22,258"
3,202504,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,20250430,"1,163,684","1,141,576","1,141,576","1,141,576","1,141,576","22,108"
4,202505,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,20250531,"1,151,558","1,129,379","1,129,379","1,129,379","1,129,379","22,179"
5,202506,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,20250630,"1,134,969","1,112,829","1,112,829","1,112,829","1,112,829","22,140"
6,202507,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,20250713,"1,126,739","1,104,655","1,104,655","1,104,655","1,104,655","22,084"


## ** Current Issue

In [20]:
''' Monthly : Prepaid Active Caller 30D Rolling : TMH (Jan-Apr 2024) '''

v_metric_cd = 'TB1S000600'

issue_pre_actv_call_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_pre_actv_call_df = issue_pre_actv_call_df.loc[issue_pre_actv_call_df['TM_KEY_MTH']>=202401]
issue_pre_actv_call_df = issue_pre_actv_call_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_pre_actv_call_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202405,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20240531,"17,391,242","17,391,242","17,190,679","17,190,679","17,190,679",0
1,202406,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20240630,"17,313,488","17,313,488","17,219,683","17,219,683","17,219,683",0
2,202407,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20240731,"17,342,927","17,342,927","17,323,115","17,323,115","17,323,115",0
3,202408,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20240831,"17,247,805","17,247,805","17,131,822","17,131,822","17,131,822",0
4,202409,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20240930,"17,190,424","17,190,424","17,178,862","17,178,862","17,178,862",0
5,202410,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20241031,"17,195,820","17,195,820","17,183,235","17,183,235","17,183,235",0
6,202411,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20241130,"17,253,443","17,253,443","17,239,155","17,239,155","17,239,155",0
7,202412,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20241231,"17,769,553","17,769,553","17,753,679","17,753,679","17,753,679",0
8,202501,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20250131,"17,344,294","17,344,294","17,330,909","17,330,909","17,330,909",0
9,202502,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,20250228,"17,424,676","17,424,676","17,407,425","17,407,425","17,407,425",0


In [21]:
''' Daily : Prepaid Active Caller 30D Rolling : TMH '''

v_metict_cd = 'TB1S000600'
# v_tm_key_mth = curr_mth
v_tm_key_mth = prev_mth

issue_daily_pre_actv_call_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_pre_actv_call_tmh_df = issue_daily_pre_actv_call_tmh_df.loc[issue_daily_pre_actv_call_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_pre_actv_call_tmh_df = issue_daily_pre_actv_call_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_pre_actv_call_tmh_df_display = issue_daily_pre_actv_call_tmh_df.copy()
mod_col_list = issue_daily_pre_actv_call_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_pre_actv_call_tmh_df_display[col] = issue_daily_pre_actv_call_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_pre_actv_call_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202507,20250714,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,nan,nan,nan,nan
1,202507,20250713,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,204,177","16,192,852","16,192,852","16,192,852"
2,202507,20250712,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,207,586","16,196,195","16,196,195","16,196,195"
3,202507,20250711,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,201,042","16,189,702","16,189,702","16,189,702"
4,202507,20250710,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,206,888","16,195,475","16,195,475","16,195,475"
5,202507,20250709,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,nan,nan,nan,nan
6,202507,20250708,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,274,115","16,262,434","16,262,434","16,262,434"
7,202507,20250707,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,317,126","16,305,046","16,305,046","16,305,046"
8,202507,20250706,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,339,704","16,326,755","16,326,755","16,326,755"
9,202507,20250705,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-07-15 04:56:33,"16,354,285","16,341,738","16,341,738","16,341,738"


In [22]:
''' Monthly : Postpaid Reported SubBase B2C (Jan-May 2024) '''

v_metric_cd = 'B2S010600'

issue_postpaid_report_sub_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_df = issue_postpaid_report_sub_b2c_df.loc[issue_postpaid_report_sub_b2c_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_df = issue_postpaid_report_sub_b2c_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202406,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20240630,0,"14,226,167","14,212,860","14,034,154","14,212,850","-14,226,167"
1,202407,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20240731,0,"14,209,033","14,195,782","14,018,425","14,195,772","-14,209,033"
2,202408,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20240831,0,"14,190,431","14,177,201","13,999,593","14,177,191","-14,190,431"
3,202409,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20240930,0,"14,186,615","14,173,403","13,996,349","14,173,393","-14,186,615"
4,202410,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20241031,0,"14,179,406","14,166,448","13,990,677","14,166,438","-14,179,406"
5,202411,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20241130,0,"14,160,522","14,154,713","13,979,080","14,154,703","-14,160,522"
6,202412,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20241231,0,"14,158,698","14,152,890","13,977,330","14,152,880","-14,158,698"
7,202501,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20250131,0,"14,069,220","14,063,387","13,889,946","14,063,360","-14,069,220"
8,202502,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20250228,0,"14,068,955","14,063,110","14,063,094","14,063,094","-14,068,955"
9,202503,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-07-15 04:56:33,20250331,0,"14,084,812","14,078,967","14,078,955","14,078,955","-14,084,812"


In [23]:
''' Monthly : Postpaid Reported SubBase B2C : TMH (Jan-May 2024) '''

v_metric_cd = 'TB2S010600'

issue_postpaid_report_sub_b2c_tmh_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_tmh_df = issue_postpaid_report_sub_b2c_tmh_df.loc[issue_postpaid_report_sub_b2c_tmh_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_tmh_df = issue_postpaid_report_sub_b2c_tmh_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_tmh_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202402,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240229,0,"8,998,482","8,998,152","8,998,139","8,998,139","-8,998,482"
1,202403,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240331,0,"8,994,853","8,987,179","8,987,169","8,987,169","-8,994,853"
2,202404,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240430,0,"8,996,646","8,989,810","8,989,800","8,989,800","-8,996,646"
3,202405,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240531,0,"9,023,324","9,009,568","9,009,558","9,009,558","-9,023,324"
4,202406,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240630,0,"9,042,184","9,041,896","9,041,886","9,041,886","-9,042,184"
5,202407,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240731,0,"9,052,569","9,052,314","9,052,304","9,052,304","-9,052,569"
6,202408,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240831,0,"9,061,002","9,060,761","9,060,751","9,060,751","-9,061,002"
7,202409,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20240930,0,"9,068,470","9,068,233","9,068,223","9,068,223","-9,068,470"
8,202410,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20241031,0,"9,080,143","9,079,912","9,079,902","9,079,902","-9,080,143"
9,202411,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,20241130,0,"9,088,656","9,088,441","9,088,431","9,088,431","-9,088,656"


In [24]:
''' Daily : Postpaid Reported SubBase B2C : TMH '''

v_metict_cd = 'TB2S010600'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_rpt_sb_b2c_tmh_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rpt_sb_b2c_tmh_df = issue_daily_post_rpt_sb_b2c_tmh_df.loc[issue_daily_post_rpt_sb_b2c_tmh_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rpt_sb_b2c_tmh_df = issue_daily_post_rpt_sb_b2c_tmh_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rpt_sb_b2c_tmh_df_display = issue_daily_post_rpt_sb_b2c_tmh_df.copy()
mod_col_list = issue_daily_post_rpt_sb_b2c_tmh_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rpt_sb_b2c_tmh_df_display[col] = issue_daily_post_rpt_sb_b2c_tmh_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rpt_sb_b2c_tmh_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202507,20250714,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,nan,nan,nan,nan
1,202507,20250713,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,084,691","9,084,449","9,084,438","9,084,438"
2,202507,20250712,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,081,234","9,080,993","9,080,982","9,080,982"
3,202507,20250711,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,087,898","9,087,655","9,087,644","9,087,644"
4,202507,20250710,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,083,650","9,083,407","9,083,396","9,083,396"
5,202507,20250709,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,nan,nan,nan,nan
6,202507,20250708,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,087,187","9,086,942","9,086,931","9,086,931"
7,202507,20250707,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,084,386","9,084,143","9,084,132","9,084,132"
8,202507,20250706,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,081,665","9,073,874","9,073,863","9,073,863"
9,202507,20250705,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-07-15 04:56:33,"9,088,110","9,087,866","9,087,855","9,087,855"


In [25]:
''' Monthly : Postpaid Reported SubBase B2C : DTAC (Jan-May 2024) '''

v_metric_cd = 'DB2S010600'

issue_postpaid_report_sub_b2c_dtac_df = monthly_df_display.loc[monthly_df_display['METRIC_CD']==v_metric_cd]
issue_postpaid_report_sub_b2c_dtac_df = issue_postpaid_report_sub_b2c_dtac_df.loc[issue_postpaid_report_sub_b2c_dtac_df['TM_KEY_MTH']>=202401]
issue_postpaid_report_sub_b2c_dtac_df = issue_postpaid_report_sub_b2c_dtac_df.sort_values(by=['TM_KEY_MTH', 'METRIC_NAME']).reset_index(drop=True)
issue_postpaid_report_sub_b2c_dtac_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202406,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20240630,0,"5,183,983","5,170,964","4,992,268","5,170,964","-5,183,983"
1,202407,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20240731,0,"5,156,464","5,143,468","4,966,121","5,143,468","-5,156,464"
2,202408,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20240831,0,"5,129,429","5,116,440","4,938,842","5,116,440","-5,129,429"
3,202409,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20240930,0,"5,118,145","5,105,170","4,928,126","5,105,170","-5,118,145"
4,202410,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20241031,0,"5,099,263","5,086,536","4,910,775","5,086,536","-5,099,263"
5,202411,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20241130,0,"5,071,866","5,066,272","4,890,649","5,066,272","-5,071,866"
6,202412,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20241231,0,"5,055,973","5,050,376","4,874,826","5,050,376","-5,055,973"
7,202501,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20250131,0,"5,033,939","5,028,342","4,854,928","5,028,342","-5,033,939"
8,202502,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20250228,0,"5,013,920","5,008,326","5,008,326","5,008,326","-5,013,920"
9,202503,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,20250331,0,"4,997,992","4,992,399","4,992,399","4,992,399","-4,997,992"


In [26]:
''' Daily : Postpaid Reported SubBase B2C : DTAC '''

v_metict_cd = 'DB2S010600'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_daily_post_rpt_sb_b2c_dtac_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_daily_post_rpt_sb_b2c_dtac_df = issue_daily_post_rpt_sb_b2c_dtac_df.loc[issue_daily_post_rpt_sb_b2c_dtac_df['METRIC_CD']==v_metict_cd]
issue_daily_post_rpt_sb_b2c_dtac_df = issue_daily_post_rpt_sb_b2c_dtac_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_daily_post_rpt_sb_b2c_dtac_df_display = issue_daily_post_rpt_sb_b2c_dtac_df.copy()
mod_col_list = issue_daily_post_rpt_sb_b2c_dtac_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_daily_post_rpt_sb_b2c_dtac_df_display[col] = issue_daily_post_rpt_sb_b2c_dtac_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_daily_post_rpt_sb_b2c_dtac_df_display

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202507,20250714,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,nan,nan,nan,nan
1,202507,20250713,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,nan,nan,nan,nan
2,202507,20250712,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,944,042","4,938,451","4,938,451","4,938,451"
3,202507,20250711,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,942,765","4,937,174","4,937,174","4,937,174"
4,202507,20250710,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,949,396",nan,nan,nan
5,202507,20250709,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,947,866",nan,nan,nan
6,202507,20250708,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,946,503","4,940,911","4,940,911","4,940,911"
7,202507,20250707,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,946,412","4,940,820","4,940,820","4,940,820"
8,202507,20250706,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,951,616","4,946,024","4,946,024","4,946,024"
9,202507,20250705,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-07-15 04:56:33,"4,950,624","4,945,033","4,945,033","4,945,033"


In [27]:
''' Monthly : TVS Active Subs '''

v_metict_cd = 'TB4S000500'
v_tm_key_mth = curr_mth
# v_tm_key_mth = prev_mth

issue_tvs_active_sub_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P', 'G', 'H', 'HH']].loc[chk_src_df['TM_KEY_MTH']>=v_tm_key_mth].copy()
issue_tvs_active_sub_df = issue_tvs_active_sub_df.loc[issue_tvs_active_sub_df['METRIC_CD']==v_metict_cd]
issue_tvs_active_sub_df = issue_tvs_active_sub_df.sort_values(by='TM_KEY_DAY', ascending=False).reset_index(drop=True)

issue_tvs_active_sub_df_display = issue_tvs_active_sub_df.copy()
mod_col_list = issue_tvs_active_sub_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    issue_tvs_active_sub_df_display[col] = issue_tvs_active_sub_df_display[col].apply(lambda x: format(x, ',.0f'))
issue_tvs_active_sub_df_display


,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,P,G,H,HH
0,202507,20250714,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,nan,nan,nan,nan
1,202507,20250713,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,104,655","1,104,655","1,104,655","1,104,655"
2,202507,20250712,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,105,528","1,105,528","1,105,528","1,105,528"
3,202507,20250711,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,105,830","1,105,830","1,105,830","1,105,830"
4,202507,20250710,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,106,376","1,106,376","1,106,376","1,106,376"
5,202507,20250709,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,107,655","1,107,655","1,107,655","1,107,655"
6,202507,20250708,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,107,849","1,107,849","1,107,849","1,107,849"
7,202507,20250707,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,109,378","1,109,378","1,109,378","1,109,378"
8,202507,20250706,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,109,378","1,109,378","1,109,378","1,109,378"
9,202507,20250705,TVS,TB4S000500,TVS Active Subs,2025-07-15 04:56:33,"1,109,696","1,109,696","1,109,696","1,109,696"
